#### 对于下载的数据，由于松辽系统年份设置太多会下载不了，所以下载以后的命名采用以下办法：命名格式为年份+入流（雨量），比如2010-入流就是2010年的实测入流数据。  本文件为提取5.1-9.30的降雨和提取6.1-9.30的径流，其中降雨多一个月是作为预热期。筛选出数据的为xlsx格式且没有年份且是倒着的。将雨量和实测入流-据转换为逐小时的，雨量数据的空缺值填充为0，实测入流数据线性插值，然后分别拼接，填充蒸发。然后重设时间，从2010.1.1.00：00开始。期间产生的一个只有雨量数据的文件将会在处理划分场次洪水数据中用到。

把原数据顺序反转，并且改为csv表格

In [3]:
import pandas as pd
import os

# 指定目录
dir_path = r'C:\Users\20676\Desktop\大伙房水库\表格'

# 获取目录下的所有Excel文件
excel_files = [f for f in os.listdir(dir_path) if f.endswith('.xlsx') or f.endswith('.xls')]

for file_name in excel_files:
    # 读取Excel文件
    df = pd.read_excel(os.path.join(dir_path, file_name))
    
    # 反转表格的行
    df = df.iloc[::-1]
    
    # 删除原Excel文件
    os.remove(os.path.join(dir_path, file_name))
    
    # 保存为CSV文件，文件名与原Excel文件相同
    new_file_name = os.path.splitext(file_name)[0] + '.csv'
    df.to_csv(os.path.join(dir_path, new_file_name), index=False)

添加年份

In [4]:
import pandas as pd
import os

# 指定目录
dir_path = r'C:\Users\20676\Desktop\大伙房水库\表格'

# 获取目录下的所有CSV文件
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]

for file_name in csv_files:
    # 读取CSV文件
    df = pd.read_csv(os.path.join(dir_path, file_name))
    
    # 获取文件名的前四个字符作为年份
    year = file_name[:4]
    
    # 将年份添加到时间列的每个值前面
    df['时间'] = year + '-' + df['时间']
    
    # 保存修改后的CSV文件
    df.to_csv(os.path.join(dir_path, file_name), index=False)

In [5]:
import pandas as pd
import os

# 获取文件夹中的所有文件名
folder_path =  r'C:\Users\20676\Desktop\大伙房水库\表格'
files = os.listdir(folder_path)

for file in files:
    # 检查文件名是否以"入流"或"雨量"结尾
    if file.endswith('入流.csv'):
        # 读取文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 保留时间列和实测入流列
        df = df[['时间', '实测入流']]
        # 删除实测入流列中的负值
        df = df[df['实测入流'] >= 0]
        # 将修改后的数据写回文件
        df.to_csv(os.path.join(folder_path, file), index=False)
    elif file.endswith('雨量.csv'):
        # 读取文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 保留时间列和面雨量列
        df = df[['时间', '面雨量']]
        # 将修改后的数据写回文件
        df.to_csv(os.path.join(folder_path, file), index=False)

In [8]:
import pandas as pd
import os

# 获取文件夹中的所有文件名
folder_path = r'C:\Users\20676\Desktop\大伙房水库\表格'
files = os.listdir(folder_path)

for file in files:
    # 检查文件名是否以"雨量"结尾
    if file.endswith('雨量.csv'):
        # 读取文件
        df = pd.read_csv(os.path.join(folder_path, file))
        # 将时间列转换为pandas的DateTime类型，并设置为索引
        df['时间'] = pd.to_datetime(df['时间'])
        df = df.set_index('时间')
        # 使用pandas的resample方法将数据重新采样为逐小时的
        df = df.resample('H').sum()
        # 使用fillna方法使用0填充面雨量列的缺失值
        df['面雨量'] = df['面雨量'].fillna(0)
        # 将修改后的数据写回文件
        df.to_csv(os.path.join(folder_path, file))

C:\Users\20676\AppData\Local\Temp\ipykernel_24952\758743190.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').sum()
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\758743190.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').sum()
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\758743190.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').sum()
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\758743190.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').sum()
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\758743190.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample('H').sum()
C:\Users\20676\AppData\Lo

In [9]:
import pandas as pd
import os

# 获取文件夹下所有以入流为结尾的csv文件
folder_path = r'C:\Users\20676\Desktop\大伙房水库\表格'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('入流.csv')]

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    
    # 读取csv文件
    df = pd.read_csv(file_path)
    
    # 将时间列转换为datetime对象
    df['时间'] = pd.to_datetime(df['时间'])
    
    # 计算重复时间戳的实测入流的平均值
    df = df.groupby('时间').mean().reset_index()
    
    # 设置时间列为索引
    df = df.set_index('时间')
    
    # 重新采样为每小时一个数据点，并对缺失值进行线性插值
    df_resampled = df.resample('H').interpolate(method='linear')
    
    # 将结果保留两位小数
    df_resampled = df_resampled.round(2)
    
    # 将处理后的数据保存回csv文件
    df_resampled.to_csv(file_path)

C:\Users\20676\AppData\Local\Temp\ipykernel_24952\2405013179.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('H').interpolate(method='linear')
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\2405013179.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('H').interpolate(method='linear')
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\2405013179.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('H').interpolate(method='linear')
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\2405013179.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('H').interpolate(method='linear')
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\2405013179.py:24: FutureWarning: '

合并雨量和入流的文件

In [12]:
import pandas as pd
import os

# 设置文件夹路径
folder_path = 'C:\\Users\\20676\\Desktop\\大伙房水库\\表格'

# 获取所有以"雨量"和"入流"结尾的文件
rainfall_files = [f for f in os.listdir(folder_path) if f.endswith('雨量.csv')]
inflow_files = [f for f in os.listdir(folder_path) if f.endswith('入流.csv')]

# 读取并合并"雨量"文件
rainfall_dfs = []
for file in rainfall_files:
    df = pd.read_csv(os.path.join(folder_path, file))
    df['时间'] = pd.to_datetime(df['时间'])
    df = df.set_index('时间')
    rainfall_dfs.append(df)
merged_rainfall_df = pd.concat(rainfall_dfs).sort_index()

# 读取并合并"入流"文件
inflow_dfs = []
for file in inflow_files:
    df = pd.read_csv(os.path.join(folder_path, file))
    df['时间'] = pd.to_datetime(df['时间'])
    df = df.set_index('时间')
    inflow_dfs.append(df)
merged_inflow_df = pd.concat(inflow_dfs).sort_index()

# 将合并后的数据保存为新的文件
merged_rainfall_df.to_csv(os.path.join(folder_path, '逐小时雨量.csv'))
merged_inflow_df.to_csv(os.path.join(folder_path, '逐小时入流.csv'))

入流的文件线性插值，雨量的文件填充0

In [14]:
import pandas as pd
import os

# 设置文件夹路径
folder_path = 'C:\\Users\\20676\\Desktop\\大伙房水库\\表格'

# 读取并处理"逐小时入流"表格
inflow_df = pd.read_csv(os.path.join(folder_path, '逐小时入流.csv'))
inflow_df['时间'] = pd.to_datetime(inflow_df['时间'])
inflow_df = inflow_df.set_index('时间')
inflow_df = inflow_df.resample('H').interpolate(method='linear').round(2)
inflow_df.to_csv(os.path.join(folder_path, '逐小时入流.csv'))

# 读取并处理"逐小时雨量"表格
rainfall_df = pd.read_csv(os.path.join(folder_path, '逐小时雨量.csv'))
rainfall_df['时间'] = pd.to_datetime(rainfall_df['时间'])
rainfall_df = rainfall_df.set_index('时间')
rainfall_df = rainfall_df.resample('H').asfreq().fillna(0)
rainfall_df.to_csv(os.path.join(folder_path, '逐小时雨量.csv'))

C:\Users\20676\AppData\Local\Temp\ipykernel_24952\275886043.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  inflow_df = inflow_df.resample('H').interpolate(method='linear').round(2)
C:\Users\20676\AppData\Local\Temp\ipykernel_24952\275886043.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rainfall_df = rainfall_df.resample('H').asfreq().fillna(0)


合并表格

In [15]:
import pandas as pd
import os

# 设置文件夹路径
folder_path = 'C:\\Users\\20676\\Desktop\\大伙房水库\\表格'

# 读取"逐小时入流"表格
inflow_df = pd.read_csv(os.path.join(folder_path, '逐小时入流.csv'))
inflow_df['时间'] = pd.to_datetime(inflow_df['时间'])
inflow_df = inflow_df.set_index('时间')

# 读取"逐小时雨量"表格
rainfall_df = pd.read_csv(os.path.join(folder_path, '逐小时雨量.csv'))
rainfall_df['时间'] = pd.to_datetime(rainfall_df['时间'])
rainfall_df = rainfall_df.set_index('时间')

# 根据时间列合并这两个表格
merged_df = pd.merge(inflow_df, rainfall_df, left_index=True, right_index=True)
merged_df.to_csv(os.path.join(folder_path, '逐小时入流和雨量.csv'))

In [16]:
import pandas as pd
import os

# 设置文件夹路径
folder_path = 'C:\\Users\\20676\\Desktop\\大伙房水库\\表格'

# 读取"逐小时入流和雨量"表格
merged_df = pd.read_csv(os.path.join(folder_path, '逐小时入流和雨量.csv'))
merged_df['时间'] = pd.to_datetime(merged_df['时间'])
merged_df = merged_df.set_index('时间')

# 创建一个布尔掩码，选择所有年份的五月一日到九月三十日的数据
mask = (merged_df.index.month >= 5) & (merged_df.index.month <= 9)
selected_df = merged_df.loc[mask]

selected_df.to_csv(os.path.join(folder_path, '逐小时入流和雨量_五月一日到九月三十日.csv'))

In [17]:
import pandas as pd
import os
import numpy as np

# 设置文件夹路径
folder_path = 'C:\\Users\\20676\\Desktop\\大伙房水库\\表格'

# 读取"逐小时入流和雨量_五月一日到九月三十日"表格
selected_df = pd.read_csv(os.path.join(folder_path, '逐小时入流和雨量_五月一日到九月三十日.csv'))
selected_df['时间'] = pd.to_datetime(selected_df['时间'])
selected_df = selected_df.set_index('时间')

# 删除全部年份的五月一日到五月三十一日的"实测入流"列的值，并填充为空
mask = (selected_df.index.month == 5) & (selected_df.index.day <= 31)
selected_df.loc[mask, '实测入流'] = np.nan

selected_df.to_csv(os.path.join(folder_path, '逐小时入流和雨量_五月一日到九月三十日.csv'))

In [18]:
#增加潜在蒸发列
import pandas as pd

# 读取CSV文件
df = pd.read_csv(f'{folder_path}\\逐小时入流和雨量_五月一日到九月三十日.csv')

# 将时间列转换为日期时间格式
df['时间'] = pd.to_datetime(df['时间'])

# 根据时间列的月份创建新的列
def calculate_pet(row):
    month = row['时间'].month
    if month == 5:
        return 99.3 / 31 / 24
    elif month == 6:
        return 102.2 / 30 / 24
    elif month == 7:
        return 100.3 / 31 / 24
    elif month == 8:
        return  92.2/ 31 / 24
    elif month == 9:
        return 72.4 / 30 / 24
    else:
        return 0

df['pet(mm/hour)'] = df.apply(calculate_pet, axis=1)

# 将新列的值四舍五入到两位小数
df['pet(mm/hour)'] = df['pet(mm/hour)'].round(3)

# 将新的DataFrame写入CSV文件
df.to_csv(f'{folder_path}\\逐小时入流雨量潜在蒸发.csv', index=False)

In [19]:
# 读取CSV文件
df = pd.read_csv(f'{folder_path}\\逐小时入流雨量潜在蒸发.csv')

# 创建新的时间列，从2010年1月1日0时起，逐行增加一个小时
df['时间'] = pd.date_range(start='2010-01-01', periods=len(df), freq='H')

# 修改列名
df = df.rename(columns={'时间': 'time', '面雨量': 'prcp(mm/hour)', '实测入流': 'flow(m^3/s)'})

# 将新的DataFrame写入CSV文件
df.to_csv(f'{folder_path}\\大伙房.csv', index=False)

C:\Users\20676\AppData\Local\Temp\ipykernel_24952\2675656083.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['时间'] = pd.date_range(start='2010-01-01', periods=len(df), freq='H')
